# Télécommande sur page Web

**Objectif:** Contrôler le rover à partir d'un site web. 

|||
:--- | :--- |
|Âge |10 à 18 ans|
|Notions abordées|Site web|
|Objectifs pédagogiques|Serveur web, page HTML|
| Durée| 2 heures|
| Dispositif pédagogiques| Par groupe de 2|
| Matériel| Un MarsCambot, Un laptop/tablette par groupe de 2, avec connexion à Internet|
| Prérequis| 1. Avoir construit le MarsCamBot et s'y être connecté avec VNC viewer (voir Activité 5 - Construction du rover). 2. Connaissances de bases de Python pour contrôler le robot (voir Activité 7 - Contrôle d'un rover avec Python) |




## Aperçu

L'activité consiste à contrôler les déplacements du rover depuis un site Web. 

## Code

In [ ]:
# Import the Time library
import time  
# Import the GPIO Zero Library
import gpiozero 

# For receiving images from the camera
import cv2

# For the Web server
import flask  

In [ ]:
# Initialise right and left motors
right_motor = gpiozero.Motor(9, 10)
left_motor = gpiozero.Motor(7, 8)

# Set motor speed
motor_speed=0.5

In [ ]:
# Function to go forward
def forward(speed=0.5, duration=1):

    # Forward: Turn both motors on with the speed "speed"
    left_motor.forward(speed)
    right_motor.forward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()

# Function to go backward
def backward(speed=0.5, duration=1):

    # Backward: Turn both motors on with the speed "-speed"
    left_motor.backward(speed)
    right_motor.backward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()

# Function to turn right    
def turn_right(speed=0.5, duration=1):

    # right: Only turn on left motor with a speed of 0.5 
    right_motor.backward()
    left_motor.forward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()
    
# Function to turn left    
def turn_left(speed=0.5, duration=1):

    # left: Only turn on left motor with a speed of 0.5 
    right_motor.forward(speed)
    left_motor.backward(speed)

    # Wait for "duration" seconds
    time.sleep(duration)

    # Turn the motors off
    left_motor.stop()
    right_motor.stop()

In [ ]:
# Display the video
def display_video():
    
    camera_object = cv2.VideoCapture(0)
    
    while True:
    
        ret, picture = camera_object.read()
        picture_rgb = cv2.cvtColor(picture, cv2.COLOR_BGR2RGB)
    
        ret, jpeg = cv2.imencode('.jpg', picture_rgb)
        pic = jpeg.tobytes()
        
        #Flask streaming
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + pic + b'\r\n\r\n')

In [ ]:
# Start Flask app (Web server)
app = flask.Flask("Remote control")

In [ ]:
# Define the control as a function of the URL
@app.route('/')
def index():
    return flask.render_template("index.html")

@app.route('/video_feed')
def video_feed():
    return flask.Response(display_video(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/forward')
def button_forward():
    forward(speed=motor_speed, duration=1)
    return ""
    
@app.route('/backward')
def button_backward():
    backward(speed=motor_speed, duration=1)
    return ""
    
@app.route('/left')
def button_left():
    turn_left(speed=motor_speed, duration=1)
    return ""
    
@app.route('/right')
def button_right():
    turn_right(speed=motor_speed, duration=1)
    return ""


In [ ]:
# Start Web server
app.run(host='0.0.0.0', port=2204, threaded=True, debug=False)